# Un nouvel estimateur basé sur scikit-learn

On souhaite construire un estimateur qui estime une régression linéaire à coefficients positifs, une autre avec des coefficients uniquement négatifs puis pour finir une dernière régression linéaire qui considère les deux premières comme features.

Une régression linéaire minimise l'erreur
$\sum_i \left\Vert X_i\theta - y_i \right\Vert^2$.
Le gradient est $\sum_i X_i'\left( X_i\theta - y_i \right)$.

Comme le modèle souhaité est équivalent à une optimisation sous contrainte,
on propose de le résoudre comme ceci :

* On applique une itération de l'algorithme de la descente de gradient :
  $\theta_{t+1} = \theta_t - \epsilon_t \sum_i X_i'\left( X_i\theta - y_i \right)$.
* On ne garde que les coefficients positifs : $\theta_{t+1} = \max(0, \theta_t)$.
* On retourne à l'étape 1 ou on s'arrête si l'algorithme a convergé.

## Le nouveau régresseur linéaire

In [20]:
import sklearn
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import StackingRegressor
from teachpyx.datasets.wines import load_wines_dataset

In [81]:
class PositiveOrNegativeLinearRegression(RegressorMixin, BaseEstimator):
    """
    Apprends une régression linéaire avec des coefficients du même signe.

    :param epsilon: gradient step
    :param max_iter: number maximum of iterations
    :param positive: only positive weights (or negative if False)
    """

    def __init__(
        self, epsilon: float = 1.0, max_iter: int = 100, positive: bool = True
    ):
        super().__init__()
        self.epsilon = epsilon
        self.max_iter = max_iter
        self.positive = positive

    def fit(self, X, y):
        "Trains."
        theta = np.random.randn(X.shape[1])
        theta = np.maximum(theta, 0) if self.positive else np.minimum(theta, 0)
        n_refresh = 0
        for i in range(self.max_iter):
            epsilon = self.epsilon * 10 / (i + 10)
            grad = X.T @ (X @ theta - y)
            theta = theta - epsilon * grad / X.shape[0]
            theta = np.maximum(theta, 0) if self.positive else np.minimum(theta, 0)
            if (
                np.abs(theta).max() == 0
                and i < self.max_iter - 5
                and n_refresh <= self.max_iter // 5
            ):
                # If all coefficients are null. The algorithm may be stuck.
                theta = np.random.randn(X.shape[1]) * epsilon
                theta = np.maximum(theta, 0) if self.positive else np.minimum(theta, 0)
                n_refresh += 1
        self.theta_ = theta
        return self

    def predict(self, X):
        "Predicts."
        if hasattr(X, "values"):
            # Input is a dataframe.
            return X.values @ self.theta_
        return X @ self.theta_

L'ordre d'héritage ``RegressorMixin, BaseEstimator`` est important pour que le modèle hérite des [Tags](https://scikit-learn.org/stable/developers/develop.html#estimator-tags). S'ils sont mal spécifiés, des *supers modèles* comme [StackingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html) créent une erreur. On vérifie que le modèle fonctionne.

In [82]:
X = np.random.randn(5, 2)
y = X[:, 0] * 0.3 + X[:, 1] * 0.6 + np.random.randn(5) / 100
lr = LinearRegression().fit(X, y)
plr = PositiveOrNegativeLinearRegression(max_iter=50, positive=True)
plr.fit(X, y)
lr.coef_, plr.theta_

(array([0.29819368, 0.60114392]), array([0.29805285, 0.60249471]))

Tout fonctionne bien.

## Assemblage

On assemble une régression linéaire à coefficients positifs, une autre à coefficients négatifs, et une dernière pour assembler le tout. On commence par estimer une régression linéaire qu'on comparera au résultat final.

In [83]:
df = load_wines_dataset()
df["color"] = df["color"].apply(lambda s: 1 if s == "red" else 0)
df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,color
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [84]:
X, y = df.drop("quality", axis=1), df["quality"]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [85]:
lr = LinearRegression(fit_intercept=False)
lr.fit(X_train, y_train)
r2_score(y_test, lr.predict(X_test))

0.2600570367886693

Si le modèle assemblé est équivalent, on devrait retrouver la même performance.

In [86]:
stacking_lr = StackingRegressor(
    estimators=[
        ("plr", PositiveOrNegativeLinearRegression()),
        ("nlr", PositiveOrNegativeLinearRegression(positive=False)),
    ],
    final_estimator=LinearRegression(fit_intercept=False),
    cv=5,  # Nombre de folds pour la validation croisée
)
stacking_lr.fit(X_train, y_train)

StackingRegressor(cv=5,
                  estimators=[('plr', PositiveOrNegativeLinearRegression()),
                              ('nlr',
                               PositiveOrNegativeLinearRegression(positive=False))],
                  final_estimator=LinearRegression(fit_intercept=False))

In [79]:
r2_score(y_test, stacking_lr.predict(X_test))

-8.554760819102542

Ca ne marche visiblement pas. Regardons les coefficients.

In [80]:
for est in stacking_lr.estimators_:
    print(est, est.theta_)

PositiveOrNegativeLinearRegression() fixed_acidity            3.850444
volatile_acidity         0.178457
citric_acid              0.171137
residual_sugar           2.908205
chlorides                0.029593
free_sulfur_dioxide     16.392868
total_sulfur_dioxide    61.588887
density                  0.531706
pH                       1.721510
sulphates                0.285379
alcohol                  5.649192
color                    0.127578
dtype: float64
PositiveOrNegativeLinearRegression(positive=False) fixed_acidity           0.0
volatile_acidity        0.0
citric_acid             0.0
residual_sugar          0.0
chlorides               0.0
free_sulfur_dioxide     0.0
total_sulfur_dioxide    0.0
density                 0.0
pH                      0.0
sulphates               0.0
alcohol                 0.0
color                   0.0
dtype: float64


## Avec un pipeline

L'in convénient de la méthode est que l'apprentissage est légèrement différent puisqu'il s'effectue avec une validation croisée. On peut supprimer ceci en créant son propre pipeline